In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spam - Data.csv")

In [3]:
df = df[["Label", "Message"]]

In [5]:
# Get a list of stop words from https://gist.github.com/sebleier/554280
with open('stopwords.txt') as file:
    lines = [line.rstrip() for line in file]

def getTopSpam(df, commonWords, num):
  '''
  Returns the most common 'num' words in the spam messages.
  '''
  spam = {}
  ham = []

  # Gets count of all words in spam that are not in the list of common words
  for idx in range(len(df.index)):
    message = df.iat[idx, 1]
    message = message.lower()
    words = message.split()
    for word in words:
      if word not in commonWords and df.iat[idx, 0] == "spam":
        if word not in spam:
          spam[word] = 1
        elif word in spam:
          spam[word] = spam[word] + 1
   
  spam = sorted(spam.items(), key=lambda x:x[1], reverse=True)
  return dict(spam[:num])

result = getTopSpam(df, lines, 5)
print(result)

{'call': 342, 'free': 180, '2': 169, 'ur': 144, 'txt': 136}


In [6]:
def vectorize (df, words):
    vectorizedData = []
    for row in range(len(df.index)):
        vectors = []
        message = df["Message"][row]
        message = message.lower()
        message = message.split()
        for word in words:
            if word in message:
                vectors.append(1)
            else:
                vectors.append(0)
        template = {"Vectors":vectors}
        vectorizedData.append(template)
    vectorizedData = pd.DataFrame.from_dict(vectorizedData)
    df = pd.concat([df, vectorizedData], ignore_index=True, axis = 1)
    return df

In [7]:
df = vectorize (df, result)

In [8]:
df = df[[0, 2]]

In [9]:
call = []
free = []
two = []
ur = []
txt = []
for index, row in df.iterrows():
    call.append(row[2][0])
    free.append(row[2][1])
    two.append(row[2][2])
    ur.append(row[2][3])
    txt.append(row[2][4])

In [10]:
scikitVectors = pd.DataFrame.from_dict({"call":call, "free":free, "2":two, "ur":ur, "txt":txt})

In [11]:
df = pd.concat([df, scikitVectors], axis = 1)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df[["call", "free", "2", "ur", "txt"]], df[0], test_size=0.3)

In [14]:
from sklearn.naive_bayes import MultinomialNB

In [15]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [16]:
predictions = clf.predict(X_test)

In [17]:
from sklearn.metrics import confusion_matrix

In [18]:
confusion_matrix(predictions, y_test.Label.values, labels=["ham", "spam"])

AttributeError: 'Series' object has no attribute 'Label'